## define libraries and costant variables

In [1]:
# merucariのスクレイピングをするプログラム
# ユーザページから商品名と商品出品日の取得をする

# install libraries
from selenium import webdriver
from selenium.webdriver.chrome import service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs4
import pandas as pd
import time
import os
import traceback
import pandas as pd
from tkinter import messagebox as mb
import re
import datetime
from dateutil.relativedelta import relativedelta
import openpyxl
import numpy

In [2]:
# define variables -  global variables
URL = 'https://jp.mercari.com/mypage/listings' # ここは変更必要
# DRIVER_PATH = 'D:\python\mercuri\driver\chromedriver.exe' # 変更必要
DRIVER_PATH = r'C:\python\mercuri\driver\chromedriver.exe' # 変更必要 - note pc用
# USERDATA_DIR = r'D:\python\mercuri\UserData'  # カレントディレクトリの直下に作る場合
USERDATA_DIR = r'C:\python\mercuri\UserData_12'  # カレントディレクトリの直下に作る場合 - note pc用 
df = pd.DataFrame() # 後でほかのファイルに渡せるようにするために変数を定義しておく
df_path = './data/selling_item.csv'

## ドライバーの設定関数

In [3]:
def get_driver():
    # driverの準備
    if os.path.exists(USERDATA_DIR):
        try:
            options = webdriver.chrome.options.Options()
            options.add_argument('--user-data-dir=' + USERDATA_DIR)
            options.page_load_strategy = 'eager'
            options.add_argument('--disable-extensions')

            # google chromeを起動
            chrome_service = service.Service(executable_path=DRIVER_PATH)
            driver = webdriver.Chrome(
                    service=chrome_service,
                    options=options
                )
            # driver.maximize_window() # 画面サイズ最大化
            driver.implicitly_wait(5) # errorが起きた場合に10秒後自動的に閉じるように設定
        except:        
            mb.showwarning('警告','EXCEPTION\n' + traceback.format_exc())
        return driver
    else:
        mb.showwarning('ログイン情報がありません','ログイン情報を取得させる必要があります！ -自動的にログインページに遷移します。')
        return 0

In [4]:
def login_mercari_first_time():
    # login情報の保持用操作 - 初期設定時のみ使用

    os.makedirs(USERDATA_DIR, exist_ok=True) # ディレクトリが存在しない場合に作成

    # option -> 使用するgoogle profileを指定
    options = webdriver.chrome.options.Options()
    options.add_argument('--user-data-dir=' + USERDATA_DIR)
    options.add_experimental_option('detach', True)

    # google chromeを起動
    chrome_service = service.Service(executable_path=DRIVER_PATH)
    driver = webdriver.Chrome(
        service=chrome_service,
        options=options
        )
    # driver.maximize_window() # 画面サイズ最大化
    driver.implicitly_wait(10) # errorが起きた場合に10秒後自動的に閉じるように設定

    driver.get('https://jp.mercari.com/mypage') # メルカリのログインページへ -> 初回利用時はログイン情報を記録させるのみ

    return driver

In [5]:
login_mercari_first_time()

<selenium.webdriver.chrome.webdriver.WebDriver (session="d8d178cab869d393c83626e69cf4df62")>

## 情報取得関数

In [5]:
def login_mercari(driver, url):
    # login情報の保持用操作

    try:
        driver.get(url)
        time.sleep(3) # 5秒後に自動的に閉じる - 変更の必要あり
        # 「もっと見る」ボタンがあればクリック
        for i in range(100):
            try:
                driver.find_element_by_xpath('//*[@id="currentListing"]/div/mer-button/button').click()
                time.sleep(1)
            except:
                break
    except:        
        mb.showwarning('警告','EXCEPTION\n' + traceback.format_exc())
    
    page_source = driver.page_source
    driver.quit()
    return page_source

In [6]:
def login_mercari_2(driver, item_urls):
    # login情報の保持用操作
    srcs = []
    i = 0
    for url in item_urls:
        try:
            # driver.get(url)
            driver.execute_script("window.open()") # 新しいタブを開く
            driver.switch_to.window(driver.window_handles[1]) # 新しいタブに切り替える
            driver.get(url)
            time.sleep(3) # 5秒後に自動的に閉じる - 変更の必要あり
            srcs.append(driver.page_source)
            driver.close()
            driver.switch_to.window(driver.window_handles[0]) # 切り替えておかないとエラーになる
            i += 1
        except:        
            mb.showwarning('警告','EXCEPTION\n' + traceback.format_exc())
    
    driver.quit()
    return srcs

In [7]:
def new_items_login_mercari(driver, item_urls, length):
    # update時に新しく情報を取得する必要があるものだけを対象にする
    srcs = []
    i = 0
    for url in item_urls:
        try:
            if i >= length:
                driver.execute_script("window.open()") # 新しいタブを開く
                driver.switch_to.window(driver.window_handles[1]) # 新しいタブに切り替える
                driver.get(url)
                time.sleep(3) # 5秒後に自動的に閉じる - 変更の必要あり
                srcs.append(driver.page_source)
                driver.close()
                driver.switch_to.window(driver.window_handles[0]) # 切り替えておかないとエラーになる
            i += 1
        except:        
            mb.showwarning('警告','EXCEPTION\n' + traceback.format_exc())
    
    driver.quit()
    return srcs

In [8]:
def get_item_urls(page_source):
    # スクレイピング実行関数

    # scrayping
    soup = bs4(page_source, features='lxml')
    item_urls = []

    try:
        # obtain elements by large category
        elems = soup.find_all('a')
        # linkのみの取得
        p = re.compile(r'/item/')
        for e in elems:
            href = e.attrs['href']
            if p.match(href):
                item_urls.append('https://jp.mercari.com' + href)
        return item_urls
    
    except Exception as e:
 
        mb.showwarning('警告','EXCEPTION\n' + traceback.format_exc())
 
        return None

In [9]:
def get_edit_urls(item_urls):
    # edit_urls = []
    # for url in item_urls:
    #    edit_urls.append(url.replace('/item/', '/sell/edit/', 1))
    edit_urls = [url.replace('/item/', '/sell/edit/', 1) for url in item_urls]
    return edit_urls

In [32]:
def find_item_info(srcs, item_urls):
    # 商品名を見つけるメソッド
    item_names = []
    item_prices = []
    item_open_days = []
    i = 0
    # scrayping
    for src in srcs:
        soup = bs4(src, features='lxml')

        try:
            # find item info space
            item = soup.find('div', id='item-info')

            # find name
            mer_heading = item.find("mer-heading")
            item_names.append(str(mer_heading.attrs['title-label']))

            # find price
            mer_price = item.find('mer-price')
            item_prices.append(int(mer_price.attrs['value']))

            # find open day
            mer_text = item.find_all('mer-text')
            p = re.compile(r'.*前$')
            tmp = []
            for text in mer_text:
                if p.search(text.text):
                    tmp.append(text.text)
            item_open_days.append(str(tmp[-1])) # 最後に入った値のみ挿入
            i += 1

        except Exception as e:
            mb.showwarning('警告','EXCEPTION\n' + traceback.format_exc())
            print(item_urls[i])
            i += 1
    
    return item_names,item_prices,item_open_days

In [11]:
def get_csvfile():
    global df 
    df.to_csv('./data/selling_item.csv', index=False)
    df.to_excel('./data/selling_item.xlsx', index=False)

In [12]:
def update_items():
    # 売れた商品は除外して、出品日数の再計算をする関数
    # 増えた商品分のみ商品詳細情報を取りに行くようにしたい
    # へんこうしないと
    global df_path
    global df 

    if os.path.exists(df_path):
        driver = get_driver()
        page_source = login_mercari(driver, URL)
        item_urls = get_item_urls(page_source)

        tmp_df = pd.read_csv(df_path)
        tmp_df2 = pd.DataFrame({'item_url':item_urls})

        merge_df = tmp_df.merge(tmp_df2, how='inner', on='item_url') # 売れた商品のそぎ落とし

        length = len(merge_df) # 次の処理時の始まりのインデックスになる
        length2 = len(item_urls)

        if not length == length2:
            # 増えた分があるときのみ実行
            srcs = new_items_login_mercari(get_driver(),item_urls,length) # 必要なインデックスから
            edit_urls = get_edit_urls(item_urls[length:]) # 必要なインデックスから
            item_names,item_prices,item_open_days = find_item_info(srcs)

            tmp = pd.DataFrame()
            tmp['item_url'] = item_urls # 出品情報
            tmp['edit_url'] = edit_urls # 出品編集ページ
            tmp['name'] = item_names # 商品名
            tmp['price'] = item_prices # 値段
            tmp['past_days'] = item_open_days # 出品日
            tmp['past_days'] = df['past_days'].replace('　', ' ')
            tmp['last_updated'] = pd.Series() # 初期時は空にする。
            tmp['selling_date'] = pd.Series() # 初期時は空にする。
            tmp['no_discount'] = pd.Series() # 初期時は空にする。
            tmp['changed_price'] = pd.Series() # 初期時は空にする。
            tmp['selling_date_is_empty'] = pd.Series() # 初期時は空にする。

            return pd.concat([merge_df,tmp], axis=0, join='inner') # 新しい要素は下に追加され
        
        return merge_df # 増えた分がないときはこれを返す。

## 情報編集関数

In [13]:
def change_mercuri_price(driver):
    # まとめて値段変更できるようにした
    global df
    try:
        for i,val in enumerate(df.values):
            if val[8]:
                driver.execute_script("window.open()") # 新しいタブを開く
                driver.switch_to.window(driver.window_handles[1]) # 新しいタブに切り替える
                driver.get(val[1])
                time.sleep(3)
                element = driver.find_element_by_name("price")
                element.send_keys(Keys.CONTROL + "a")
                element.send_keys(Keys.DELETE)
                element.send_keys(str(val[3]))
                driver.find_element_by_xpath('//*[@id="main"]/form/div[2]/mer-button[1]/button').click()
                driver.close()
                driver.switch_to.window(driver.window_handles[0]) # 切り替えておかないとエラーになる
    except:        
        mb.showwarning('警告','EXCEPTION\n' + traceback.format_exc())
    df['changed_price'] = False # 値段の変更が完了後はすべてのフラグをfalseに変えておく
    df['no_discount'] = True

In [14]:
def apply_discount(All='Y'): # 変更必要
    # 期日過ぎているのを一括で値下げ
    # 選べるようにしたいならまたここを変える
    All = All.upper()
    new_price = []
    changed = []
    selling_date = []
    if All == 'Y':
        # 1週間たっているitemすべて値引き
        for i,val in enumerate(df.values):
            if not val[7]:
                if val[3] >= 2000: # 2000円以上→200円引き
                    val[3] = val[3] - int(200)
                elif 1000 <= val[3] <= 2000: # 1000円以上2000円以下→100円引き
                    val[3] = val[3] - int(100)
                else: # その他10円引き
                    val[3] = val[3] - int(10)
                new_price.append(val[3])
                changed.append(True)
                selling_date.append(datetime.date.today())
                
            else:
                new_price.append(val[3])
                changed.append(False)
                selling_date.append(None)
    return new_price,changed,selling_date # 新しい値段、変更フラグ、販売日を返す


In [62]:
def discount_needed():
    # 値下げが必要かのフラグを付与する関数
    # True 必要なし、False 必要
    global df
    tmp = []
    for i,val in enumerate(df.values):
        if val[9]: # datetimeの出品日がない場合はスクレイピングで取得したものを使用する
           if re.match('[1-6] 日前|.*秒前|.*分前|.*時間前|.*時前',val[4]):
               tmp.append(True)
           else:
                tmp.append(False)
        else:
            if datetime.datetime.strptime(val[6], '%Y-%m-%d') > datetime.timedelta(3): # 6日以上離れてたら値引き必要にする
                tmp.append(False)
            else:
                tmp.append(True)
    return tmp

## 値取得関数

In [16]:
def get_df():
    return df

## 実行関数

In [59]:
def get_item_info():
    # item情報をとるための関数をすべてじっこうする
    # 初期実行時のみ使用
    global df
    global df_path
    # 各itemのurlの取得
    print('各商品の詳細情報のurlを取得中！')
    driver = get_driver()
    page_source = login_mercari(driver, URL)
    item_urls = get_item_urls(page_source)
    edit_urls = get_edit_urls(item_urls)
    print('DONE!')
    # return item_urls

    # 詳細情報の取得
    driver = get_driver()
    srcs = login_mercari_2(driver, item_urls)
    print('item info done')

    # return srcs
    item_names, item_prices, item_open_days = find_item_info(srcs,item_urls)
    df['item_url'] = item_urls # 出品情報
    df['edit_url'] = edit_urls # 出品編集ページ
    df['name'] = item_names # 商品名
    df['price'] = item_prices # 値段
    df['past_days'] = item_open_days # 出品日
    df['last_updated'] = datetime.date.today() # 更新日
    # discountが必要かの判断
    df['past_days'] = df['past_days'].replace('　', ' ')
    df['selling_date'] = pd.Series() # 初期時は空にする。
    df['no_discount'] = pd.Series() # 初期時は空にする。
    df['changed_price'] = False
    df['selling_date'] = selling_date()
    df['selling_date_is_empty'] = df['selling_date'].isnull() # selling_dateがあるかどうか
    df['no_discount'] = discount_needed()
    get_csvfile()
    print('DONE!')

In [18]:
def excute_discount():
    # 値下げを実行するための関数
    global df_path
    df = pd.read_csv(df_path) # でーたを読み込み
    new_price,changed,sell_date = apply_discount(All='Y')
    df['price'] = new_price
    df['changed_price'] = changed
    df['selling_date'] = sell_date
    df['selling_date_is_empty'] = df['selling_date'].isnull() # selling_dateがあるかどうか
    change_mercuri_price(get_driver())

In [37]:
def excute_update():
    # update時はこれを参照
    global df

    df = update_items()

    df['last_updated'] = datetime.date.today()
    df['no_discount'] = discount_needed()
    df['selling_date_is_empty'] = df['selling_date'].isnull() # selling_dateがあるかどうか
    df['selling_date'] = selling_date()
    df['changed_price'] = False

    get_csvfile()

In [56]:
def open_excel():
    # excelを開く
    import subprocess

    EXCEL = r'./data/selling_item.xlsx'
    subprocess.Popen(['start',EXCEL], shell=True)

## 日付の設定

In [20]:
def selling_date():
    # 7日以上たっているものは値下げされる前提なので特に付与しない
    global df 
    tmp = []
    for i,val in enumerate(df.values):
        p = re.compile(r'1 日前|.*秒前|.*分前|.*時間前|.*時前')
        if p.match(val[4]):
            tmp.append(datetime.date.today() - datetime.timedelta(days=1))
        elif re.match('2 日前', val[4]):
            tmp.append(datetime.date.today() - datetime.timedelta(days=2))
        elif re.match('3 日前',val[4]):
            tmp.append(datetime.date.today() - datetime.timedelta(days=3))
        elif re.match('4 日前',val[4]):
            tmp.append(datetime.date.today() - datetime.timedelta(days=4))
        elif re.match('5 日前',val[4]):
            tmp.append(datetime.date.today() - datetime.timedelta(days=5))
        elif re.match('6 日前',val[4]):
            tmp.append(datetime.date.today() - datetime.timedelta(days=6))
        else:
            tmp.append(None)
    return tmp

## test logic

while(True):
    ans = str(input('ログインしたことありますか？(はい→Y いいえ→N): '))
    ans = ans.upper()
    if ans == 'N':
        login_mercari_first_time()
    if ans == 'Y':
        get_item_info()
        break
    else:
        print('Y か N で答えてください')
        continue

In [33]:
# item = ['https://jp.mercari.com/item/m35524357866']
# srcs = login_mercari_2(get_driver(), item)
# item_names, item_prices, item_open_days = find_item_info(srcs,item)

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 666 entries, 0 to 665
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   item_url               666 non-null    object
 1   edit_url               666 non-null    object
 2   name                   666 non-null    object
 3   price                  666 non-null    int64 
 4   past_days              666 non-null    object
 5   last_updated           666 non-null    object
 6   selling_date           647 non-null    object
 7   no_discount            666 non-null    bool  
 8   changed_price          666 non-null    bool  
 9   selling_date_is_empty  666 non-null    bool  
dtypes: bool(3), int64(1), object(6)
memory usage: 43.6+ KB


In [63]:
type(datetime.date.today() - datetime.timedelta(days=3))

datetime.date

In [65]:
datetime.datetime.strptime(df['selling_date'][0], '%Y-%m-%d')

TypeError: strptime() argument 1 must be str, not datetime.date

In [66]:
datetime.datetime.strptime(str(datetime.date.today() - datetime.timedelta(days=3)), r'%Y-%m-%d')

datetime.datetime(2022, 1, 12, 0, 0)

In [69]:
datetime.datetime.strptime(str(df['selling_date'][0]), r'%Y-%m-%d') 

datetime.datetime(2022, 1, 11, 0, 0)

In [73]:
datetime.datetime.strptime(str(df['selling_date'][0]), r'%Y-%m-%d') > datetime.datetime.strptime(str(datetime.date.today() - datetime.timedelta(days=3)), r'%Y-%m-%d')

True

In [80]:
tmp = []
for i,val in enumerate(df.values):
    if not val[9]:
        print(val[6])
        tmp.append((datetime.datetime.strptime(str(val[6]), r'%Y-%m-%d') >= datetime.datetime.strptime(str(datetime.date.today() - datetime.timedelta(days=3)), r'%Y-%m-%d')))
tmp

2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11
2022-01-11

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


In [10]:
df['a'] = pd.Series()
df['b'] = [5]
df

C:\Users\USER_A~1\AppData\Local\Temp/ipykernel_14588/1818357921.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['a'] = pd.Series()


,a,b
0,NaN,5


In [22]:
len(str(df['a'][0]))
if numpy.isnan('21'):
    print(True)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''